<a href="https://colab.research.google.com/github/farukemrekaraca/ztm-tensorflow-machinelearning-deeplearning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!nvidia-smi

Thu Feb  1 09:13:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0              28W /  70W |    135MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-02-01 09:13:09--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-02-01 09:13:10 (97.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [20]:
from helper_functions import *

In [21]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2024-02-01 09:13:10--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.175.207, 74.125.24.207, 142.251.10.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.175.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  24.1MB/s    in 66s     

2024-02-01 09:14:16 (23.6 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [22]:
unzip_data("101_food_classes_10_percent.zip")

In [23]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [24]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/apple_pie'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/red_velvet_cake'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/pancakes'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/mussels'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/caesar_salad'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/lasagna'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/oysters'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '101_food_clas

In [25]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [26]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         monitor="val_accuracy")

In [27]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
], name="data_augmentation")

In [28]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False

input = layers.Input(shape=(224, 224, 3))
x = data_augmentation(input)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
output = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)

model = tf.keras.Model(input, output)

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                             

In [30]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [31]:
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                          epochs=5,
                                          validation_data=test_data,
                                          validation_steps=int(0.15 * len(test_data)),
                                          callbacks=[checkpoint_callback])

Epoch 1/5
237/237 [==============================] - 207s 824ms/step - loss: 3.5393 - accuracy: 0.2343 - val_loss: 2.6196 - val_accuracy: 0.4264
Epoch 2/5
237/237 [==============================] - 136s 571ms/step - loss: 2.4354 - accuracy: 0.4438 - val_loss: 2.1919 - val_accuracy: 0.4873
Epoch 3/5
237/237 [==============================] - 108s 454ms/step - loss: 2.0519 - accuracy: 0.5149 - val_loss: 1.9488 - val_accuracy: 0.5201
Epoch 4/5
237/237 [==============================] - 88s 369ms/step - loss: 1.8402 - accuracy: 0.5589 - val_loss: 1.8205 - val_accuracy: 0.5363
Epoch 5/5
237/237 [==============================] - 84s 352ms/step - loss: 1.6905 - accuracy: 0.5846 - val_loss: 1.7858 - val_accuracy: 0.5424


In [32]:
fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

790/790 [==============================] - 61s 77ms/step - loss: 1.6174 - accuracy: 0.5749


[1.6174368858337402, 0.5748910903930664]

In [33]:
model.load_weights("101_classes_10_percent_data_model_checkpoint")

In [34]:
model.evaluate(test_data)

790/790 [==============================] - 53s 67ms/step - loss: 1.6174 - accuracy: 0.5749


[1.6174368858337402, 0.5748910903930664]

In [35]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False

input = layers.Input(shape=(224, 224, 3))
x = data_augmentation(input)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
output = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)

test_model = tf.keras.Model(input, output)

In [36]:
test_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [37]:
test_model.evaluate(test_data)

790/790 [==============================] - 60s 72ms/step - loss: 4.7153 - accuracy: 0.0072


[4.7153496742248535, 0.007247524801641703]

In [38]:
test_model.load_weights("101_classes_10_percent_data_model_checkpoint")

In [39]:
test_model.evaluate(test_data)

790/790 [==============================] - 54s 68ms/step - loss: 1.6174 - accuracy: 0.5749


[1.6174368858337402, 0.5748910903930664]

Model can evidently be recovered and fitting phase can be skipped after google colab session is reset if the weights have been saved. ModelCheckpoint can be used towards this goal. The checpoint files generated by the code must be downloaded to local computer or saved to drive and in the next session the model only needs to be compiled and then the weights loaded from the saved file, which can be uploaded from local drive/google drive to google colab.